<a href="https://colab.research.google.com/github/shaggy99999/hse21_hw3/blob/main/hse21_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовительный этап



### Установка HISAT2 (для выравниваия RNA-seq чтений на геном)




In [ ]:
!apt-get update

In [ ]:
!apt-get install hisat2

In [ ]:
!hisat2 --version

### Установка sra-toolkit (для скачивания .fastq файлов из NCBI)

In [ ]:
!apt-get install sra-toolkit

### Установка FastQC

In [ ]:
!java -version

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip

In [ ]:
!unzip fastqc_v0.11.9.zip

In [12]:
!chmod a+x FastQC/fastqc

In [ ]:
!./FastQC/fastqc --help

### Установка HTSeq-count

In [ ]:
!pip install HTSeq

In [15]:
!htseq-count --version

0.13.5


### Скачиваем RNA-seq данные


*   Перепрограммированные образцы: SRR3414629, SRR3414630, SRR3414631
*   Контрольные образцы: SRR3414635, SRR3414636, SRR3414637





In [ ]:
!time  fastq-dump  --split-files  SRR3414629

In [ ]:
!time  fastq-dump  --split-files  SRR3414630

In [ ]:
!time  fastq-dump  --split-files  SRR3414631

In [ ]:
!time  fastq-dump  --split-files  SRR3414635

In [ ]:
!time  fastq-dump  --split-files  SRR3414636

In [ ]:
!time  fastq-dump  --split-files  SRR3414637

### Скачиваем геном мыши mm10 (проиндексированный для HISAT2)

In [ ]:
!wget https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz

In [ ]:
!tar -xzvf mm10_genome.tar.gz

### Скачиваем аннотацию генов GENCODE для генома мыши mm10

In [ ]:
!wget http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.annotation.gtf.gz

In [25]:
!gzip -d gencode.vM25.annotation.gtf.gz

# Выравнивание чтений

Ниже приведены примеры запуска только для одного файла. Вам следует сделать это для всех 6-ти файлов.

*   Для каждого образца запускаем FastQC. Скачиваем полученные html файлы и проверяем есть ли проблемы с файлами. Можно также установить multiqc и получить объединенный .html файл. Полученные результаты приводим в отчете на Github-е.



In [ ]:
!./FastQC/fastqc  /content/SRR3414629_1.fastq

In [ ]:
!./FastQC/fastqc  /content/SRR3414630_1.fastq

In [ ]:
!./FastQC/fastqc  /content/SRR3414631_1.fastq

In [ ]:
!./FastQC/fastqc  /content/SRR3414635_1.fastq

In [ ]:
!./FastQC/fastqc  /content/SRR3414636_1.fastq

In [ ]:
!./FastQC/fastqc  /content/SRR3414637_1.fastq



*   Запускаем HISAT2 и картируем все чтения на геном мыши.
*   Общее кол-во чтений, и кол-во, которое удалось успешно откартировать (уникально и не уникально), указано в файле SRR3414636.hisat. Приводим эту информацию в отчете на Github



In [32]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414636_1.fastq -S SRR3414636_1.sam  2>  SRR3414636.hisat


real	10m52.779s
user	17m35.042s
sys	1m3.171s


In [33]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414629_1.fastq -S SRR3414629_1.sam  2>  SRR3414629.hisat


real	1m3.514s
user	0m0.501s
sys	0m3.791s


In [34]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414630_1.fastq -S SRR3414630_1.sam  2>  SRR3414630.hisat


real	0m2.765s
user	0m0.316s
sys	0m2.422s


In [35]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414631_1.fastq -S SRR3414631_1.sam  2>  SRR3414631.hisat


real	0m2.195s
user	0m0.263s
sys	0m1.913s


In [36]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414635_1.fastq -S SRR3414635_1.sam  2>  SRR3414635.hisat


real	0m2.075s
user	0m0.233s
sys	0m1.828s


In [37]:
!time hisat2 -p 3 -x mm10/genome -U SRR3414637_1.fastq -S SRR3414637_1.sam  2>  SRR3414637.hisat


real	0m2.120s
user	0m0.237s
sys	0m1.863s


In [44]:
!cat SRR3414636.hisat

20307147 reads; of these:
  20307147 (100.00%) were unpaired; of these:
    550088 (2.71%) aligned 0 times
    17825380 (87.78%) aligned exactly 1 time
    1931679 (9.51%) aligned >1 times
97.29% overall alignment rate


In [39]:
!cat SRR3414629.hisat

(ERR): hisat2-align died with signal 2 (INT) 


In [40]:
!cat SRR3414630.hisat

(ERR): hisat2-align died with signal 2 (INT) 


In [41]:
!cat SRR3414631.hisat

(ERR): hisat2-align died with signal 2 (INT) 


In [42]:
!cat SRR3414635.hisat

(ERR): hisat2-align died with signal 2 (INT) 


In [43]:
!cat SRR3414637.hisat

(ERR): hisat2-align died with signal 2 (INT) 




*   Отбираем только те чтения, которые откартировались уникально (флаг NH:i:1):



In [ ]:
!grep -P '^@|NH:i:1$' SRR3414636_1.sam > SRR3414636.uniq.sam

In [ ]:
!grep -P '^@|NH:i:1$' SRR3414629_1.sam > SRR3414629.uniq.sam

In [ ]:
!grep -P '^@|NH:i:1$' SRR3414630_1.sam > SRR3414630.uniq.sam

In [ ]:
!grep -P '^@|NH:i:1$' SRR3414631_1.sam > SRR3414631.uniq.sam

In [ ]:
!grep -P '^@|NH:i:1$' SRR3414635_1.sam > SRR3414635.uniq.sam

In [ ]:
!grep -P '^@|NH:i:1$' SRR3414637_1.sam > SRR3414637.uniq.sam



*   Удаляем исходные .sam файлы, т.к. они нам больше не потребуются:




In [ ]:
!rm -v SRR3414636_1.sam

In [ ]:
!rm -v SRR3414629_1.sam

In [ ]:
!rm -v SRR3414630_1.sam

In [ ]:
!rm -v SRR3414631_1.sam

In [ ]:
!rm -v SRR3414635_1.sam

In [ ]:
!rm -v SRR3414637_1.sam



*   Считаем кол-во уникально-картированных чтений (17825380 чтений)



In [ ]:
!grep -v '^@' SRR3414636.uniq.sam | wc -l

In [ ]:
!grep -v '^@' SRR3414629.uniq.sam | wc -l

In [ ]:
!grep -v '^@' SRR3414630.uniq.sam | wc -l

In [ ]:
!grep -v '^@' SRR3414631.uniq.sam | wc -l

In [ ]:
!grep -v '^@' SRR3414635.uniq.sam | wc -l

In [ ]:
!grep -v '^@' SRR3414637.uniq.sam | wc -l



*   С помощью программы HTSeq Подсчитываем количество чтений, попавших на каждый ген (для одного образца программа работает 15-20 мин):
*   Аннтоация генома мыши в формате .gtf находится в файле gencode.vM25.annotation.gtf



In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414636.uniq.sam  gencode.vM25.annotation.gtf > SRR3414636.counts

In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414629.uniq.sam  gencode.vM25.annotation.gtf > SRR3414629.counts

In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414630.uniq.sam  gencode.vM25.annotation.gtf > SRR3414630.counts

In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414631.uniq.sam  gencode.vM25.annotation.gtf > SRR3414631.counts

In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414635.uniq.sam  gencode.vM25.annotation.gtf > SRR3414635.counts

In [ ]:
!time htseq-count --format=sam --stranded=no SRR3414637.uniq.sam  gencode.vM25.annotation.gtf > SRR3414637.counts



*   Смотрим сколько чтений не удалось приписать ни одному гену:
*   __no_feature 1332692 – столько чтений соответствует участкам генома, где не аннотировано ни одного экзона
*   __ambiguous 735108 – столько чтений могут принадлежать разным генам
*   Более подробно про такие случаи можно посмотреть в описании HTSeq-count - https://htseq.readthedocs.io/en/release_0.11.1/count.html
*   Итого, общее число чтений, соответствующих хотя бы одному гену равно: 17825380 – 1332692 – 735108 = 15757580
Для каждого образца приводим эту статистику в отчете на Github



In [ ]:
! grep '^__' SRR3414636.counts

In [ ]:
! grep '^__' SRR3414629.counts

In [ ]:
! grep '^__' SRR3414630.counts

In [ ]:
! grep '^__' SRR3414631.counts

In [ ]:
! grep '^__' SRR3414635.counts

In [ ]:
! grep '^__' SRR3414637.counts

Объединям все файлы .counts по генам в один общий файл ALL.counts.
В качестве заголовка для каждого образца указывать не его SRR id, а c1, c2, c3 для контрольных образцов и r1, r2, r3 для перепрограммированных образцов.
Загружаем полученный ALL.counts файл в Github репозиторий